In [ ]:
from dataset_train import FetalBrainDataset, preprocess
from torchvision.utils import make_grid
from torchvision.io import read_image
from matplotlib import pyplot as plt
import torchsummary
from network import UNet
import torch

In [ ]:
images_folder = 'data/testing_data/images/'
masks_folder = 'data/test_results'
masks_folder_gt = 'data/testing_data/masks/'
dataset = FetalBrainDataset(images_folder, masks_folder, img_size=224)
dataset_gt = FetalBrainDataset(images_folder, masks_folder_gt, img_size=224)

In [ ]:
model = UNet()
model.load_state_dict(torch.load('models/model_checkpoint_epoch_0.pt'))

In [ ]:
model.eval()
output_img = model(dataset[0][0].unsqueeze(0)).squeeze()

In [57]:
import torchio as tio
import numpy as np
image = tio.ScalarImage("data/testing_data/images/img_0000.nii.gz")
prediction = np.zeros(image.shape[1:])

for test_data_idx in range(image.shape[-1]):
    img_array = tio.ScalarImage(tensor=image.data[:, :, :, None, test_data_idx].float(),
                                affine=image.affine)

    img_array = preprocess(img_array, img_size=224, intensity=True)
    inputs = img_array['data'][:, None, :, :, 0]
    outputs = model(inputs)
    outputs = torch.argmax(torch.softmax(outputs, dim=1), dim=1)

    label_array = tio.LabelMap(tensor=outputs[:, :, :, None].cpu().long(), affine=img_array.affine)
    label_array = preprocess(label_array, img_size=image.shape[1], intensity=False)

    out = label_array['data'].squeeze().cpu().detach().numpy()
    prediction[:, :, test_data_idx] = out

torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])


In [ ]:
prediction.shape

In [ ]:
for img in image:
    img = img.squeeze()
    plt.imshow(img,'cmap=gray')
    plt.show()

In [ ]:
img1, img2 = output_img
img1 = img1.detach().numpy()
img2 = img2.detach().numpy()
plt.imshow(img1,cmap='gray')
plt.show()
plt.imshow(img2,cmap='gray')
plt.show()

In [ ]:
img1.shape

In [ ]:
idx = 0
image, label = dataset[idx]
_, label_gt = dataset_gt[idx]
image_numpy = image.squeeze().numpy()
mask_numpy = label.squeeze().numpy()
label_gt_numpy = label_gt.squeeze().numpy()
plt.imshow(image_numpy, cmap='gray')
plt.show()
plt.imshow(mask_numpy, cmap='gray')
plt.show()
plt.imshow(label_gt_numpy, cmap='gray')
plt.show()